In [1]:
# Cell 1: Load and convert Fashion MNIST dataset to a DataFrame
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import DataLoader, TensorDataset

### Pretend we are NOT doing this - imagine it being you donwloading data from the internet

In [2]:
# Download and load the Fashion MNIST dataset
transform = transforms.ToTensor()  # Transform the data into tensors
full_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)

In [ ]:
# Convert the dataset to a DataFrame
# Each image is a 28x28 array, flatten it into a single row
data_list = []
for image, label in full_dataset:
    flat_image = image.view(-1).numpy()  # Flatten the 28x28 image into a 1D array
    data_list.append([*flat_image, label])

# Create a DataFrame
columns = [f"pixel_{i}" for i in range(28 * 28)] + ["label"]
full_df = pd.DataFrame(data_list, columns=columns)

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(full_df, test_size=0.2, random_state=42)

# Display the first few rows of each DataFrame
print("Train DataFrame:")
print(train_df.head())
print("\nTest DataFrame:")
print(test_df.head())

### Let's start here by examining the data

In [ ]:
train_df = pd.DataFrame(data_list, columns=columns)

# Display the first few rows of the DataFrame
train_df.head()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.max()

## Helper to visualize a row of data - nice for images

In [ ]:
# Cell 8: Function to display a greyscale image from a row
def show_image_from_row(row, label):
    # Reshape the row back to a 28x28 image
    image = np.array(row).reshape(28, 28)
    
    # Plot the image
    plt.imshow(image, cmap="gray")
    plt.title(f"Label: {label}")
    plt.axis("off")
    plt.show()

# Example: Display an image from the DataFrame
example_row = train_df.iloc[0, :-1]  # First row (pixels only)
example_label = train_df.iloc[0, -1]  # Corresponding label
show_image_from_row(example_row, example_label)


## Example Dataloader

In [ ]:

# Define a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        """
        Initialize the dataset by passing a DataFrame.
        Args:
            dataframe (pd.DataFrame): DataFrame with pixel data and labels.
        """
        self.data = dataframe.iloc[:, :-1].values  # Features (all columns except the last)
        self.labels = dataframe.iloc[:, -1].values  # Labels (last column)

    def __len__(self):
        """
        Return the total number of samples in the dataset.
        """
        return len(self.labels)

    def __getitem__(self, idx):
        """
        Retrieve a single sample from the dataset.
        Args:
            idx (int): Index of the sample to retrieve.
        Returns:
            (torch.Tensor, torch.Tensor): Tuple of features and label.
        """
        # Convert the features and label to tensors
        features = torch.tensor(self.data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return features, label



In [ ]:
# Function to split the DataFrame and create DataLoaders
def create_train_test_dataloaders(dataframe, test_size=0.2, batch_size=64, shuffle=True):
    """
    Split the DataFrame into train and test sets and create DataLoaders.
    Args:
        dataframe (pd.DataFrame): DataFrame containing the data.
        test_size (float): Proportion of the data to be used as test data.
        batch_size (int): Number of samples per batch.
        shuffle (bool): Whether to shuffle the data.
    Returns:
        tuple: (train_loader, test_loader)
    """
    # Split the data into train and test sets
    train_df, test_df = train_test_split(dataframe, test_size=test_size, random_state=42)
    
    # Create train and test datasets
    train_dataset = CustomDataset(train_df)
    test_dataset = CustomDataset(test_df)
    
    # Create DataLoaders for train and test sets
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

In [ ]:
# ## command + /

# # Example usage: Creating train and test DataLoaders
# train_loader, test_loader = create_train_test_dataloaders(train_df, test_size=0.2, batch_size=32)

# # Example: Check the first batch from the train DataLoader
# for batch_idx, (features, labels) in enumerate(train_loader):
#     print(f"Train Batch {batch_idx + 1}")
#     print("Features shape:", features.shape)
#     print("Labels shape:", labels.shape)
#     break  # Display only the first batch

# # Example: Check the first batch from the test DataLoader
# for batch_idx, (features, labels) in enumerate(test_loader):
#     print(f"Test Batch {batch_idx + 1}")
#     print("Features shape:", features.shape)
#     print("Labels shape:", labels.shape)
#     break  # Display only the first batch

## Manually Build dataloaders

In [ ]:
# Cell 2: Create a DataLoader


# Convert the DataFrame back to PyTorch tensors
X_train = torch.tensor(train_df.iloc[:, :-1].values, dtype=torch.float32)  # All pixels
y_train = torch.tensor(train_df["label"].values, dtype=torch.long)  # Labels
# Convert the DataFrame back to PyTorch tensors
X_test = torch.tensor(test_df.iloc[:, :-1].values, dtype=torch.float32)  # All pixels
y_test = torch.tensor(test_df["label"].values, dtype=torch.long)  # Labels

# Create a TensorDataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create a DataLoader for batching and shuffling
batch_size = 64  # Number of samples per batch
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Example: Check the first batch
for batch in train_loader:
    images, labels = batch
    print(f"Batch size: {images.shape}, Labels: {labels.shape}")
    break
# Example: Check the first batch
for batch in test_loader:
    images, labels = batch
    print(f"Batch size: {images.shape}, Labels: {labels.shape}")
    break

In [ ]:
# Cell 3: Define the neural network

# Define a feedforward network for classification
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.relu = nn.ReLU()  # ReLU activation
        self.fc2 = nn.Linear(hidden_size, num_classes)  # Output layer for classification

    def forward(self, x):
        x = self.fc1(x)  # First layer
        x = self.relu(x)  # Activation
        x = self.fc2(x)  # Output layer
        return x

# Initialize the model
input_size = 28 * 28  # 784 input features (flattened image)
hidden_size = 128  # Number of neurons in the hidden layer
num_classes = 10  # 10 classes for Fashion MNIST

In [ ]:
model = SimpleNN(input_size, hidden_size, num_classes)

# Print the model architecture
print(model)

In [ ]:
# Cell 4: Define hyperparameters
learning_rate = 0.001
num_epochs = 10
batch_size = 64

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


### for later!

In [ ]:
# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )
# model.to("cpu")
# print(f"Using {device} device")


## Model Training

In [ ]:
# Training the model
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()  # Compute gradients
        optimizer.step()  # Update weights

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")


## Saving and loading a model

In [ ]:
# Save the model after training
torch.save(model.state_dict(), "fashion_mnist_model.pth")
print("Model saved as 'fashion_mnist_model.pth'")


In [ ]:
# Load the saved model for inference
model = SimpleNN(input_size, hidden_size, num_classes)  # Reinitialize the model structure
model.load_state_dict(torch.load("fashion_mnist_model.pth"))
model.eval()  # Set the model to evaluation mode
print("Model loaded and ready for inference.")


In [ ]:
# Cell 6: Inference loop
# Running inference
model.eval()  # Set the model to evaluation mode (disables dropout, etc.)
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation for efficiency
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}%")


In [ ]:

# Generate predictions for the entire dataset
all_labels = []
all_predictions = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy())

# Compute and plot the confusion matrix
cm = confusion_matrix(all_labels, all_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(range(10)))
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


### Inspect a row of data

In [ ]:
# Helper function to run inference on a single row of data
def run_inference_on_row(row, label, model):
    # Convert the row into a tensor
    row_tensor = torch.tensor(row.values, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

    # Perform inference
    with torch.no_grad():
        logits = model(row_tensor)  # Raw logits
        probabilities = torch.softmax(logits, dim=1)  # Apply softmax to convert logits to probabilities

    # Get predicted label
    predicted_label = torch.argmax(probabilities).item()

    # Print details
    print("Logits:", logits.numpy())
    print("Softmax Probabilities:", probabilities.numpy())
    print("Correct Label:", label)
    print("Predicted Label:", predicted_label)

    return predicted_label

# Example: Run inference on a single row
example_row = test_df.iloc[10, :-1]  # 10th row (pixels only)
example_label = test_df.iloc[10, -1]  # Corresponding label
run_inference_on_row(example_row, example_label, model)

In [ ]:
### Display examples of correct and incorrect predictions

In [ ]:

def display_examples(data_df, model, correct=True, num_examples=3):
    count = 0
    plt.figure(figsize=(10, 5))

    for idx, row in data_df.iterrows():
        if count >= num_examples:
            break

        # Extract image and label
        image_row = row[:-1]
        true_label = row[-1]

        # Run inference
        predicted_label = run_inference_on_row(image_row, true_label, model)

        # Check if prediction is correct
        if (predicted_label == true_label and correct) or (predicted_label != true_label and not correct):
            # Plot the image
            plt.subplot(1, num_examples, count + 1)
            image = image_row.values.reshape(28, 28)  # Reshape to 28x28
            plt.imshow(image, cmap="gray")
            plt.title(f"True: {true_label}, Pred: {predicted_label}")
            plt.axis("off")
            count += 1






In [ ]:
# Example: Show 3 correct predictions
print("Examples of Correct Predictions:")
display_examples(train_df, model, correct=True, num_examples=3)
plt.show()

In [ ]:
# Example: Show 3 incorrect predictions
print("Examples of Incorrect Predictions:")
display_examples(train_df, model, correct=False, num_examples=3)
plt.show()